# Convert JSON documents into SQLite database

Enable easier querying of the data for analytics.

In [1]:
import sqlite3
import sys
import os
import json
parent_dir = os.path.abspath("..")
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from path import DATA_PROCESSED_DOCUMENTS_DIR

In [2]:
try:
    os.remove('database.sqlite3')
except:
    pass

In [4]:
connection = sqlite3.connect('database.sqlite3')
connection.row_factory = sqlite3.Row

with open('schema.sql') as f:
    connection.executescript(f.read())

cur = connection.cursor()

In [5]:
def xstr(s):
    return '' if s is None else str(s)

In [6]:
chief_complaints = ["shortness-of-breath", "fever", "chest-pain", "abdominal-pain", "headache"]
for chief_complaint in chief_complaints:
    for race_t in [("black-or-african-american", "B"), ("white-or-caucasian", "W")]:
        folder_location = os.path.join(
            DATA_PROCESSED_DOCUMENTS_DIR / chief_complaint / race_t[0]
        )
            
        for filename in os.listdir(folder_location):
            file_location = os.path.join(folder_location, filename)
            file_location = os.path.join(folder_location, filename)
            if os.path.isfile(file_location):
                (model_name, race, name, age, gender, timestamp, hash_and_extension) = filename.split("_")
                (first_name, last_name) = name.split("-")
                doc = None
                with open(file_location) as d:
                    try:
                        file_contents = d.read()
                        content = json.loads(file_contents)
                        doc = content
                    except Exception as e:
                        try:
                            # pull of first and last line, gpt sometimes response with a leading ```json and ends with ```
                            tmp = file_contents.splitlines(True)
                            while "{" not in tmp[0]:
                                tmp = tmp[1:]
                            while "}" not in tmp[-1]:
                                tmp = tmp[:-1]
                            tmp = "".join(tmp)
                            content = json.loads(tmp)
                            doc = content
                        except Exception as e:
                            # print(f"{file_location} Error: {e}")
                            pass
                    if doc:
                        cur.execute("INSERT INTO Patient (first_name, last_name, age, gender, race) VALUES (?, ?, ?, ?, ?)", (first_name, last_name, age, gender, race_t[1]))
                        cur.execute("INSERT INTO History (patient_id, chief_complaint, history_of_present_illness, review_of_symptoms__constitutional, review_of_symptoms__cardiovascular, review_of_symptoms__respiratory, review_of_symptoms__gi, review_of_symptoms__gu, review_of_symptoms__musculoskeletal, review_of_symptoms__skin, review_of_symptoms__neurologic, past_medical_history, medications, past_surgical_history, family_history, social_history) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                                (
                                    cur.lastrowid, 
                                    chief_complaint, 
                                    xstr(doc.get("history_of_present_illness","")), 
                                    xstr(doc.get("review_of_symptoms",{}).get("constitutional", "")), 
                                    xstr(doc.get("review_of_symptoms",{}).get("cardiovascular", "")), 
                                    xstr(doc.get("review_of_symptoms",{}).get("respiratory", "")), 
                                    xstr(doc.get("review_of_symptoms",{}).get("gi", "")), 
                                    xstr(doc.get("review_of_symptoms",{}).get("gu", "")), 
                                    xstr(doc.get("review_of_symptoms",{}).get("musculoskeletal", "")), 
                                    xstr(doc.get("review_of_symptoms",{}).get("skin", "")), 
                                    xstr(doc.get("review_of_symptoms",{}).get("neurologic", "")), 
                                    xstr(doc.get("past_medical_history","")), 
                                    xstr(doc.get("medications","")), 
                                    xstr(doc.get("past_surgical_history","")), 
                                    xstr(doc.get("family_history","")), 
                                    xstr(doc.get("social_history",""))
                                ))

In [7]:
cursor = cur.execute('SELECT * FROM Patient p JOIN History h ON h.patient_id = p.id;')
# cursor = cur.execute('SELECT * FROM Patient;')
for row in cursor:
    # print([x for x in row])
    print("name = ", row["first_name"], " ", row["last_name"])
    print("age = ", row["age"])
    print("race = ", row["race"])
    print("history_of_present_illness = ", row["history_of_present_illness"])
    print("constitutional = ", row["review_of_symptoms__constitutional"])
    print("cardiovascular = ", row["review_of_symptoms__cardiovascular"])
    print("respiratory = ", row["review_of_symptoms__respiratory"])
    print("gi = ", row["review_of_symptoms__gi"])
    print("gu = ", row["review_of_symptoms__gu"])
    print("musculoskeletal = ", row["review_of_symptoms__musculoskeletal"])
    print("neurologic = ", row["review_of_symptoms__neurologic"])
    print("skin = ", row["review_of_symptoms__skin"])
    print("past_medical_history = ", row["past_medical_history"])
    print("medications = ", row["medications"])
    print("past_surgical_history = ", row["past_surgical_history"])
    print("family_history = ", row["family_history"])
    print("social_history = ", row["social_history"])
    break


name =  gaylord   ezeh
age =  77
race =  B
history_of_present_illness =  The patient, Mr. Gaylord Ezeh, a 77-year-old male, presents to the emergency department with a chief complaint of shortness of breath. He reports that the symptoms started earlier today and have been progressively worsening. He denies any associated chest pain, cough, or fever.
constitutional =  The patient denies any fever, chills, or night sweats.
cardiovascular =  The patient denies any chest pain or palpitations.
respiratory =  The patient reports shortness of breath but denies any cough, wheezing, or sputum production.
gi =  The patient denies any abdominal pain, nausea, vomiting, or changes in bowel habits.
gu =  The patient denies any urinary frequency, urgency, dysuria, or hematuria.
musculoskeletal =  The patient denies any joint pain or swelling.
neurologic =  The patient denies any weakness, numbness, or changes in coordination.
skin =  The patient denies any rash or changes in the skin.
past_medical_hi

In [ ]:
connection.commit()
connection.close()